#Statoil Iceberg

This notebook demonstrates how to download data using the Kaggle API, augment image data, and build a CNN for the [Statoil/C-CORE Iceberg Classifier Challenge](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge).  



##Downloading the Data using the Kaggle API

Install the Kaggle library to allow you to use the Kaggle API.  Download your credentials, "kaggle.json", from the Kaggle website.  The following instructions to download your kaggle credentials to your local computer:



1.   Log in to Kaggle.com.  This should take you to home hub.
2.   Click the picture of your profile on the top right corner of your home hub.  Select My Account.
3.   In your account settings, find API and click on the Create New API Token button.  This should automatically download your credentials to your computer.










In [0]:
#install Kaggle library
!pip install Kaggle

After running the following code, follow the link to receive a verification code.  Copy the code and paste it in the textbox that appears.  Ensure that the credential is located in the correct folder.  If your credential is placed in your google drive or google cloud storage, ensure that the credential is located in the correct folder.  In the following code below, I need to create a folder in my google drive called ".kaggle" and place my credential in that folder.

```
filename = "/root/.kaggle/kaggle.json"
```



In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download the files from the Statoil/C-Core Iceberg Classifier Challenge using the Kaggle API.  An easy way to get the correct command is to go to the [data page of the competition](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge/data) and copy the command located to the right of the data sources.

In [0]:
!kaggle competitions download -c statoil-iceberg-classifier-challenge

Unzip the downloaded files.  Note that for this Kaggle competition, the data is compressed in the 7z format rather than the Zip format. 

In [0]:
import os

os.system( '7z e train.json.7z' )
os.system( '7z e test.json.7z' )

##Data Preparation

The data can be normalized and augmented before being used to train the model.


In [0]:
#import python libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2 

In [0]:
train = pd.read_json("train.json")
test = pd.read_json("test.json")

Reshape the HH band and HV band for the CNN.  Then normalize the pixel values for the bands.

In [0]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())

        imgs.append(np.dstack((a, b)))

    return np.array(imgs)

In [0]:
X_train = get_scaled_imgs(train)

Retrieve the labels from the training data.

In [0]:
Y_train = np.asarray(train['is_iceberg'])

Augment images to increase the size of the dataset.  Each image in the data set is flipped vertically and horizontally.

In [0]:
def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        
        vert_flip_imgs.append(np.dstack((av, bv)))
        hori_flip_imgs.append(np.dstack((ah, bh)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [0]:
X_train_augmented = get_more_images(X_train) 
Y_train_augmented = np.concatenate((Y_train,Y_train,Y_train))

Separate training data into training and evaluation data.

In [0]:
X_eval = X_train_augmented[3850:]
Y_eval = Y_train_augmented[3850:]
X_train_final = X_train_augmented[:3850]
Y_train_final = Y_train_augmented[:3850]


Convert data to be in float32 for better speed when training the model.

In [0]:
X_train_final = X_train_final.astype(np.float32)
Y_train_final = Y_train_final.astype(np.float32)
X_eval = X_eval.astype(np.float32)
Y_eval= Y_eval.astype(np.float32)

##TensorFlow CNN Model

In [0]:
#reset graph if a previous graph has been created
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


reset_graph()

Set the parameters for the CNN.

In [0]:
learning_rate = 0.001
n_epochs = 50 
num_input = 75*75 
num_classes = 2 
dropout = 0.2 

Build the graph.  The graph contains 4 convolution layers, 2 dense layers, and a logit layer.

In [0]:
X = tf.placeholder(tf.float32, shape=(None, 75, 75, 2), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")


with tf.variable_scope('ConvNet'):

    he_init = tf.contrib.layers.variance_scaling_initializer()

    # Convolution layer #1 - 64 filters and a kernel size of 3
    conv1 = tf.layers.conv2d(X, filters=64,  kernel_size=[3, 3], activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], strides=2)
    dropout1 = tf.layers.dropout(pool1, rate = dropout)
    
    # Convolution layer #2 - 128 filters and a kernel size of 3 
    conv2 = tf.layers.conv2d(dropout1, filters=128,  kernel_size=[3,3], activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2, 2], strides=2)
    dropout2 = tf.layers.dropout(pool2, rate = dropout)

    # Convolution layer #3 - 128 filters and a kernel size of 3 
    conv3 = tf.layers.conv2d(dropout2, filters=128, kernel_size=[3,3], activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(conv3, pool_size=[2, 2], strides=2)
    dropout3 = tf.layers.dropout(pool3, rate = dropout)

    # Convolution layer #4 - 64 filters and a kernel size of 3 
    conv4 = tf.layers.conv2d(dropout3, filters=64, kernel_size=[3,3], activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling2d(conv4, pool_size=[2, 2], strides=2)
    dropout4 = tf.layers.dropout(pool4, rate = dropout)
    
    # Flatten the data to a 1-D vector for the fully connected layer
    fc1 = tf.contrib.layers.flatten(dropout4)

    # Fully connected layer #1
    fc2 = tf.layers.dense(fc1, 512, 
                        kernel_initializer=he_init, activation=tf.nn.relu)
    dropout_fc2 = tf.layers.dropout(fc2, rate = dropout)

    # Fully connected layer #2
    fc3 = tf.layers.dense(dropout_fc2, 256, 
                        kernel_initializer=he_init, activation=tf.nn.relu)
    dropout_fc3 = tf.layers.dropout(fc3, rate = dropout)


    logits = tf.layers.dense(dropout_fc3, num_classes, activation=tf.nn.sigmoid)

In [0]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [0]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [0]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
print('training model\n')
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: X_train_final, y: Y_train_final})   
        acc_train = accuracy.eval(feed_dict={X: X_train_final, y: Y_train_final})
        acc_test = accuracy.eval(feed_dict={X: X_eval,
                                            y: Y_eval})
    
        print(epoch, "Train accuracy:", acc_train, "Validation accuracy:", acc_test)
    save_path = saver.save(sess, "./cam_iceberg_model_final.ckpt")